<a href="https://colab.research.google.com/github/tranvohuy/Markovify_sentence_Truyen_Kieu/blob/master/Vanilla_char_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this note we investigate a LSTM way to produce new characters. There are two codes, 
- https://gist.github.com/vinhkhuc/7ec5bf797308279dc587: pure Tensorflow code
- http://karpathy.github.io/2015/05/21/rnn-effectiveness/: keras layer code

Both are based on this https://github.com/oxford-cs-ml-2015/practical6/blob/master/train.lua 

In [0]:
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlopen

def crawling_oneweb(url):
  html = urlopen(url).read()
  soup = BeautifulSoup(html)

  main_body = soup.findAll("p", {"class": "Normal"})
  del main_body[-1]
  text = "".join([p.text for p in main_body])
  return text  

def crawlingwebs(urls):
  '''
  Input:
  urls: a list of urls
  
  Output:
  a combination text extracted from url in urls
  '''
  text = ''
  for url in urls:
    text = text + crawling_oneweb(url)
  return text
  
urls = ["https://vnexpress.net/goc-nhin/nguoi-giau-va-thien-tai-3808335.html",
       'https://vnexpress.net/goc-nhin/thoat-khoi-co-don-3886302.html',
       'https://vnexpress.net/goc-nhin/lam-luat-3900478.html',
       'https://vnexpress.net/goc-nhin/thuat-san-dat-vang-3879380.html',
       'https://vnexpress.net/goc-nhin/nhung-mua-xuan-trong-doi-3877957.html',
       'https://vnexpress.net/goc-nhin/than-phan-ruong-dong-3862785.html',
       'https://vnexpress.net/goc-nhin/long-dan-3824537.html',
       'https://vnexpress.net/goc-nhin/ro-rang-voi-dat-3764906.html',
       'https://vnexpress.net/goc-nhin/thuong-nho-dong-bang-3778190.html']

text = crawlingwebs(urls)

# pure tensorflow

In [0]:
"""
Vanilla Char-RNN using TensorFlow by Vinh Khuc (@knvinh).
Adapted from Karpathy's min-char-rnn.py
https://gist.github.com/karpathy/d4dee566867f8291f086
Requires tensorflow>=1.0
BSD License
"""
import random
import numpy as np
import tensorflow as tf

seed_value = 42
tf.set_random_seed(seed_value)
random.seed(seed_value)

#np.eye(vocab_size) is the unit matrix of dimension vocab_size x vocab_size

def one_hot(v):
    return np.eye(vocab_size)[v]

data = text
chars = sorted(list(set(text))) # the set of different characters in the text
data_size, vocab_size = len(data), len(chars)
#vocab_size means char_size
#data_size: total characters in the originial text

print('Data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

# Hyper-parameters
hidden_size   = 100  # hidden layer's size
seq_length    = 25   # number of characters for each sample (input)
learning_rate = 1e-1

inputs     = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="inputs")
targets    = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

initializer = tf.random_normal_initializer(stddev=0.1)

with tf.variable_scope("RNN") as scope:
    hs_t = init_state
    ys = []
    for t, xs_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0: scope.reuse_variables()  # Reuse variables
        Wxh = tf.get_variable("Wxh", [vocab_size, hidden_size], initializer=initializer)
        Whh = tf.get_variable("Whh", [hidden_size, hidden_size], initializer=initializer)
        Why = tf.get_variable("Why", [hidden_size, vocab_size], initializer=initializer)
        bh  = tf.get_variable("bh", [hidden_size], initializer=initializer)
        by  = tf.get_variable("by", [vocab_size], initializer=initializer)

        hs_t = tf.tanh(tf.matmul(xs_t, Wxh) + tf.matmul(hs_t, Whh) + bh)
        ys_t = tf.matmul(hs_t, Why) + by
        ys.append(ys_t)

hprev = hs_t
output_softmax = tf.nn.softmax(ys[-1])  # Get softmax for sampling

outputs = tf.concat(ys, axis=0)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

# Minimizer
minimizer = tf.train.AdamOptimizer()
grads_and_vars = minimizer.compute_gradients(loss)

# Gradient clipping
grad_clipping = tf.constant(5.0, name="grad_clipping")
clipped_grads_and_vars = []
for grad, var in grads_and_vars:
    clipped_grad = tf.clip_by_value(grad, -grad_clipping, grad_clipping)
    clipped_grads_and_vars.append((clipped_grad, var))

# Gradient updates
updates = minimizer.apply_gradients(clipped_grads_and_vars)

# Session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# Initial values
n, p = 0, 0
hprev_val = np.zeros([1, hidden_size])

while True:
    # Initialize
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev_val = np.zeros([1, hidden_size])
        p = 0  # reset

    # Prepare inputs
    input_vals  = [char_to_ix[ch] for ch in data[p:p + seq_length]]
    target_vals = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]
    
    input_vals  = one_hot(input_vals)
    target_vals = one_hot(target_vals)

    hprev_val, loss_val, _ = sess.run([hprev, loss, updates],
                                      feed_dict={inputs: input_vals,
                                                 targets: target_vals,
                                                 init_state: hprev_val})
    if n % 500 == 0:
        # Progress
        print('iter: %d, p: %d, loss: %f' % (n, p, loss_val))

        # Do sampling
        sample_length = 200
        start_ix      = random.randint(0, len(data) - seq_length)
        print('star_ix', start_ix)
        sample_seq_ix = [char_to_ix[ch] for ch in data[start_ix:start_ix + seq_length]]
        ixes          = []
        sample_prev_state_val = np.copy(hprev_val)

        for t in range(sample_length):
            sample_input_vals = one_hot(sample_seq_ix)
            sample_output_softmax_val, sample_prev_state_val = \
                sess.run([output_softmax, hprev],
                         feed_dict={inputs: sample_input_vals, init_state: sample_prev_state_val})
            #np.darray.ravel() return a flat array
            ix = np.random.choice(range(vocab_size), p=sample_output_softmax_val.ravel())
            
            ixes.append(ix)
            sample_seq_ix = sample_seq_ix[1:] + [ix]

        txt = ''.join(ix_to_char[ix] for ix in ixes)
        print('----\n %s \n----\n' % (txt,))

    p += seq_length
    n += 1

In [0]:
input_vals  = [char_to_ix[ch] for ch in data[p:p + seq_length]]
target_vals = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]
print(input_vals)
print(target_vals)

[1, 49, 94, 59, 53, 1, 53, 55, 47, 59, 1, 65, 63, 66, 76, 59, 9, 1, 26, 60, 59, 1, 122, 58, 1]
[49, 94, 59, 53, 1, 53, 55, 47, 59, 1, 65, 63, 66, 76, 59, 9, 1, 26, 60, 59, 1, 122, 58, 1, 92]


# Keras

- https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

- https://keras.io/layers/recurrent/#lstm

In [0]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('One hot encoding...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

#optimizer = RMSprop(lr=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=optimizer)
#we can write 
#model.compile(loss='categorical_crossentropy', optimizer = 'RMSprop')
#but then it won't give the option to choose learning rate of RMSprop. 
#the default lr of RMSprop is lr=0.001

#However we can write
model.compile(loss='categorical_crossentropy', optimizer = RMSprop(lr=0.01))

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    #start_index = random.randint(0, len(text) - maxlen - 1)
    start_index=0 # choose a fixed seed sentence
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        #choose a random sentence in sentences
        sentence = text[start_index: start_index + maxlen]
        
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        #print is just a wrapper of sys.stdout.write
        sys.stdout.write(generated)
        
        #print(generated)
        
        #produce a number of new character
        for i in range(400):
            #one hot encoding of sentence
            x_pred = np.zeros((1, maxlen, len(chars)))
            
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
               
            # without [0] the shape of preds is (1,147) not (147,)
            preds = model.predict(x_pred, verbose=0)[0]
            # preds is a probability density from [0,146]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            #update sentence to predict the next character
            sentence = sentence[1:] + next_char
      
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print() #new line

print_callback = LambdaCallback(on_epoch_end = on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

corpus length: 50088
total chars: 147
nb sequences: 16683
Vectorization...
Build model...
Epoch 1/60
16683/16683 [==============================] - 23s 1ms/step - loss: 2.8559

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "Tôi còn nhớ một lời thoại trong vở kịch "
Tôi còn nhớ một lời thoại trong vở kịch thông thông thác thông thiến thiện thông thược thông thông thông thông thược thiến thông đất thông thông thược thông thiện thông thông thông thông thiến thiện thông thông thông thông thông thác thông thiện thiện thiến thông thông bất thông thông thông thông thông thông thông thông thông thông thông thông thông thông thiện thông thông thác thiến thông thông thông thông thông thông thông thang thiện
----- diversity: 0.5
----- Generating with seed: "Tôi còn nhớ một lời thoại trong vở kịch "
Tôi còn nhớ một lời thoại trong vở kịch tông thất trong thông đất được thiện, thông vào chược thiện tông hược vớ thống thưng, chàng được thiện đới than công trác

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in log


 ạô phải trường và công thức đường gên bổế chon cho đơi nữa. Thó khí nhà ba, công gó cản nghệ ảnh chính còn nguyên đã đang chúng thẳng phải "công xó. ác ý còn bằnn bất con bao đất đai tính cũng sông làm lử. áì ciền cảnh lý hiến hài
----- diversity: 1.2
----- Generating with seed: "Tôi còn nhớ một lời thoại trong vở kịch "
Tôi còn nhớ một lời thoại trong vở kịch ốt. Người lập chứy cácn trường để đóch sa, hẳnh nói hành khỏ không thêu xuyết định tiền đã họố chốm trờng đất đất" là ciống nhất lặc, sát cma đấtớnôm t: gcảm về 5xm mọ Từ ngheo được sông có hện trọi gọc lại cam dẫm có số sử dụng cũng khi hình ôn đJ đầu là khí lại, không đất được huan bất đang đưa khi nhiêu sử tới đây, cỡc bồn vùng thay diền đất để chố nam từ để lần đổi ụ khí hạc lên. Ngườ ta yề bị
Epoch 11/60
16683/16683 [==============================] - 22s 1ms/step - loss: 1.1015

----- Generating text after Epoch: 10
----- diversity: 0.2
----- Generating with seed: "Tôi còn nhớ một lời thoại trong vở kịch "
Tôi còn nhớ một l

In [0]:
print(sentences[0:10])
print(next_chars)

['Tôi còn nhớ một lời thoại trong vở kịch ', ' còn nhớ một lời thoại trong vở kịch "Hồ', 'n nhớ một lời thoại trong vở kịch "Hồn T', 'hớ một lời thoại trong vở kịch "Hồn Trươ', 'một lời thoại trong vở kịch "Hồn Trương ', ' lời thoại trong vở kịch "Hồn Trương Ba,', 'i thoại trong vở kịch "Hồn Trương Ba, da', 'hoại trong vở kịch "Hồn Trương Ba, da hà', 'i trong vở kịch "Hồn Trương Ba, da hàng ', 'rong vở kịch "Hồn Trương Ba, da hàng thị']
['"', 'n', 'r', 'n', 'B', ' ', ' ', 'n', 't', 't', 'c', ' ', ' ', 'c', 'i', 'L', ' ', 'a', ' ', '.', 'h', 'v', 'ô', ' ', 'ư', 'g', 'a', 'u', ' ', 'ậ', 'r', 'c', 'n', 'm', 'h', 'ú', 'm', 'h', ' ', 'g', 'ã', 'h', ' ', 'o', 'h', ' ', 'c', 'n', 'h', 'g', 'h', ',', 'à', 'ỏ', ' ', 'ế', 'l', 'ô', ' ', 'ì', 'ã', 'n', ' ', 'm', 'ô', 'v', ' ', 'g', 'ã', 'ố', ' ', 'i', 'h', ' ', 'ư', 'h', 'n', '?', '\n', 'g', 'r', 'n', 'B', 't', ' ', 'i', 'ằ', ':', 'T', ' ', 'i', 'à', 'ã', 'ố', ' ', 'i', 'h', ' ', ' ', 'm', 'h', ' ', 'n', 'ã', 's', ' ', 'à', 'n', ' ', 'n', 'ư', ' 

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               141312    
_________________________________________________________________
dense_2 (Dense)              (None, 147)               18963     
Total params: 160,275
Trainable params: 160,275
Non-trainable params: 0
_________________________________________________________________


In [0]:
y[0,:]

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,